# Init

In [ ]:
import pandas as pd
import numpy as np
import json

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
FILE_PATH = '.\\data\\ratebeer.json'
# TODO temporarily using smaller dataset so I can analyze results faster
FILE_PATH = '.\\data\\beer_50000.json'

f = open(FILE_PATH)
dataset = []
for l in f:
    dataset.append(eval(l))

In [ ]:
# DESCRIPTIONS ARE ACCORDING TO CHATGPT (take with a grain of salt)


ales = set() # ALE
ipas = set() # A subset of ales
dark_ales = set() # A subset of ales

lagers = set() # LAGERs are different somehow? idek man

sakes = set() # RICE BEER
misc = set() #Beers I was unable to classify into a category

# I don't know if this is a 'real' category but these are all fruit infused or based beers
fruitbeers = set()

# Not sure how to categorize the 'lambics'
# MEAD IS ITS OWN CATEGORY OF ALCOHOL BUT I PUT IT INTO p_fruit cuz its sweet

for d in dataset:
    # presets based on ChatGPT categorizations
    p_ales = ['Belgian White &#40;Witbier&#41;', 'Berliner Weisse', 'Bire de Garde', 'Saison', 'Schwarzbier', 'Abbey Dubbel', 'Abbey Tripel', 'Abt/Quadrupel', 'Altbier', 'Barley Wine', 'German Hefeweizen', 'German Kristallweizen', 'Weizen Bock']
    p_paleales = ['Premium Bitter/ESB', 'Bitter',]
    p_lagers = ['Dunkelweizen', 'Kölsch', 'Klsch', 'Eisbock', 'Malt Liquor', 'California Common', 'Doppelbock', 'Dunkel', 'Dunkler Bock', 'Dortmunder/Helles', 'Heller Bock', 'Oktoberfest/Mrzen', 'Vienna', 'Zwickel/Keller/Landbier']
    p_fruit = ['Cider', 'Fruit Beer', 'Ice Cider/Perry', 'Perry', 'Mead']

    if not d:
        continue
    s = d['beer/style']
    if 'Pale Ale' in s or 'IPA' in s or s in p_paleales:
        ipas.add(s)
    elif 'Ale' in s or s in p_ales:
        ales.add(s)
    elif 'Stout' in s or 'Porter' in s:
        dark_ales.add(s)
    elif 'Lager' in s or 'Pilsener' in s or s in p_lagers:
        lagers.add(s)
    elif 'Sak' in s:
        sakes.add(s)
    elif s in p_fruit:
        fruitbeers.add(s)
    else:
        misc.add(s)

# print the size of each set
print('Ales:', len(ales))
print('Dark Ales:', len(dark_ales))
print('IPAs:', len(ipas))
print('Sakes:', len(sakes))
print('Fruit Beers:', len(fruitbeers))

print(misc)

# TODO
'''
It is definitely worth looking into splitting up the 'ale' dataset into more specific categories.
I haven't looked into the dataset enough but I assume the vast majority of beers fall under this category, 
unless we break it up or perform some model balancing I would expect it to be heavily biased towards this category.

The same could possibly be said fo for the lagers category, but I'm not sure.

If you are still stuck with an extremely low accuracy even with this tweak, I think it'd be a good idea that
we look into other features. I still believe that identifying certain keywords in the text part of the review
would work extremely well for this case.
'''
''

Ales: 29
Dark Ales: 8
IPAs: 7
Sakes: 12
Fruit Beers: 5
{'Lambic - Faro', 'Lambic - Gueuze', 'Smoked', 'Lambic - Fruit', 'Low Alcohol', 'Specialty Grain', 'Spice/Herb/Vegetable', 'Lambic - Unblended'}


''

In [63]:
misc

{'Bohemian Pilsener',
 'Classic German Pilsener',
 'Lambic - Faro',
 'Lambic - Fruit',
 'Lambic - Gueuze',
 'Lambic - Unblended',
 'Low Alcohol',
 'Pilsener',
 'Smoked',
 'Specialty Grain',
 'Spice/Herb/Vegetable'}

In [ ]:
#TODO this isn't parsing right when using ratebeer.json
# It throws a KeyError for styles that are not
'''
('India Pale Ale &#40;IPA&#41;', 185953)
('Pale Lager', 145618)
('Belgian Strong Ale', 123946)
('Imperial Stout', 122223)
('Imperial/Double IPA', 105672)
'''
k = 'beer/style'
styles = {}
for d in dataset:
    try:
        s = d[k]
    except KeyError:
        continue
    if s in styles:
        styles[s] +=1
    else:
        styles[s] = 1

# sort styles
sorted_styles = sorted(styles.items(), key=lambda x: x[1], reverse=True)
for k in sorted_styles[:5]:
    print(k)

#TODO For proper classification we should probably group these together in some way?
# I don't know how we would go about this, if it is as straightforward as 'all IPAs go into one group'

('India Pale Ale &#40;IPA&#41;', 185953)
('Pale Lager', 145618)
('Belgian Strong Ale', 123946)
('Imperial Stout', 122223)
('Imperial/Double IPA', 105672)


In [ ]:
X, y = [], []
for d in dataset:
    f1 = d['review/appearance']
    f2 = d['review/palate']
    f3 = d['review/taste']
    f4 = d['review/aroma']
    X.append([f1, f2, f3, f4])

    t = d['beer/style']
    y.append(t)


In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)